In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
import os
import parsers as p
import vector_parser_api as vp
from multiprocessing import Pool
import multiprocessing
from pathlib import Path
import scipy.io
import time
import json
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def add_to_dir(root, directory, data):
    cwd = root
    for index, i in enumerate(directory[:-1]):
        if i not in cwd.keys():
            cwd[i] = {}
        cwd = cwd[i]
    cwd[directory[-1]] = data

def str_to_directory(s):
    s=s.replace("]","")
    l = re.split('[.]|[\/]|[\\]|[\[]', s)
    return [int(x) if x.isdigit() else x for x in l]


timea=time.time()
df = pd.read_csv("../telemetry_exe/Raw_Data/data0005.csv", dtype = str, on_bad_lines = "skip")
init_sz = df.shape[0]
df = df[df["time"].str.len()==13]
df = df[df['msg.len'].str.len()==1]
df = df[~df['msg.id'].isnull()]
df = df[~(df['msg.id'].str.len()<2)]
df = df[~(df['msg.id'].str.len()>3)]
#df['time'] = pd.to_datetime(df['time'], unit='ms')
df['time'] = pd.to_numeric(df['time'])
df["msg.id"] = df["msg.id"].apply(int, base = 16)
df["msg.len"] = pd.to_numeric(df['msg.len'])
df = df[df['msg.len']*2 == df['data'].str.len()]
df["data"] = df["data"].apply(int, base = 16)
df["data"] = df['data'].to_numpy(dtype=np.uint64).byteswap()>>(8*(8-df["msg.len"])).to_numpy(dtype=np.uint16)
df = df.reset_index(drop=True)
pruned_sz = df.shape[0]
print("Removed messages: " + str(init_sz - pruned_sz))
timeb = time.time()
print("CSV parse time: " + str(timeb-timea))
msg_id = df['msg.id'].to_numpy().astype(np.uint16)
msg_data = df['data'].to_numpy()
msg_time = df['time'].to_numpy()
points = 0
messages = 0
root = {}
for i in p.MESSAGE_DICT.values():
    try:
        outputs = i[0](msg_data, msg_id, msg_time)
        messages += 1
        points += outputs[0][0].shape[0]
        for data in outputs:
            for idx, d in enumerate(data[3]):
                add_to_dir(root, str_to_directory(d), data[0][:, (0,idx+1)])
    except:
        continue
#print(messages)
#print(points)
print("Message coverage: "+str(points/msg_id.size))
print("Message parse time: " + str(time.time()-timeb))
print("Total time: " + str(time.time()-timea))
print("Speed: " + str(init_sz/(time.time()-timea)))

Removed messages: 77
CSV parse time: 0.5252792835235596
Message coverage: 0.9931872595264821
Message parse time: 0.01657581329345703
Total time: 0.5428550243377686
Speed: 284053.7401088059


In [98]:
%%prun
filename = "../telemetry_exe/Raw_Data/data0005.csv"
root = vp.parse_file(filename)
#print(json.dumps(root, sort_keys=True, indent=4, default=str))
#scipy.io.savemat('data.mat', {"data": root})

         325872 function calls (325593 primitive calls) in 1.440 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.325    0.162    0.325    0.162 {pandas._libs.lib.maybe_convert_numeric}
        6    0.296    0.049    0.297    0.049 {pandas._libs.lib.map_infer_mask}
   308277    0.199    0.000    0.199    0.000 apply.py:141(f)
        2    0.198    0.099    0.397    0.199 {pandas._libs.lib.map_infer}
        1    0.101    0.101    0.103    0.103 {method 'read_low_memory' of 'pandas._libs.parsers.TextReader' objects}
        8    0.047    0.006    0.047    0.006 {built-in method pandas._libs.missing.isnaobj}
        6    0.038    0.006    0.038    0.006 {pandas._libs.algos.take_2d_axis1_object_object}
        6    0.020    0.003    0.020    0.003 {pandas._libs.lib.infer_dtype}
       35    0.015    0.000    0.015    0.000 {built-in method numpy.empty}
        7    0.011    0.002    0.011    0.002 {method 'take' 

In [36]:
detailed_voltage = root["BMS"]["detailed_voltage"]
mask = (detailed_voltage["group"][:,1]==1) & (detailed_voltage["ic"][:,1]==10)
detailed_voltage["voltage"]["2"][:,1][mask]

array([4.0363, 4.0362, 4.0362, 4.0362, 4.0362, 4.0361, 4.0362, 4.0362,
       4.0362, 4.0363, 4.0361, 4.0362, 4.0362, 4.0362, 4.0362, 4.0362,
       4.0362, 4.0362, 4.0363, 4.0363, 4.0363, 4.0362, 4.0362, 4.0363,
       4.0362, 4.0362, 4.0362, 4.0362, 4.0361, 4.0362, 4.036 , 4.0362,
       4.0361, 4.0362, 4.0362, 4.0363, 4.0362, 4.0362, 4.0361, 4.0363,
       4.0362, 4.0362, 4.0362, 4.0362, 4.0362, 4.0363, 4.0362, 4.0362,
       4.0362, 4.0359, 4.0295, 4.0035, 3.8558, 3.8532, 3.9803, 3.9722,
       4.0481, 4.0117, 3.9782, 4.0183, 3.989 , 3.8605, 3.7356, 3.9667,
       4.0901, 4.0395, 4.0506, 4.0273, 4.    , 4.0038, 3.9881, 3.9643,
       3.9876, 4.0555, 4.0033, 3.994 , 3.9747, 3.8941, 3.9907, 4.003 ,
       4.0189, 4.0165, 3.9998, 4.0014, 3.9979, 3.9856, 4.0079, 4.0147,
       4.0099, 4.0102, 4.011 , 4.0112, 4.0123, 4.0124, 3.9949, 3.9648,
       3.9685, 3.9824, 4.0002, 4.0063, 4.0068, 4.0074, 4.0078, 4.0045,
       4.0151, 4.0101, 4.0115, 4.0117, 4.0121, 4.0122, 4.0124, 4.0129,
      

In [90]:
"{arr:02}".format(arr=10)

'10'

In [199]:
df = df.sample(frac=1).reset_index(drop=True)
df.sort_values(by=['time'],inplace=True)
df = df.reset_index(drop=True)

In [53]:
msg_id = df['msg.id'].to_numpy().astype(np.uint16)
msg_data = df['data'].to_numpy()

In [3]:
unique, freq = np.unique(msg_id, return_counts = True)
hexes = np.array([p.MESSAGE_DICT[i][1] for i in unique])
print(np.array((freq,hexes)).T)

[['36366' 'MC1_status']
 ['35320' 'MC2_status']
 ['35719' 'MC3_status']
 ['35866' 'MC4_status']
 ['34911' 'MC1_temps']
 ['34759' 'MC2_temps']
 ['34847' 'MC3_temps']
 ['34755' 'MC4_temps']
 ['9194' 'MC1_energy']
 ['8694' 'MC2_energy']
 ['8899' 'MC3_energy']
 ['8992' 'MC4_energy']
 ['9237' 'MC1_setpoints_command']
 ['8805' 'MC2_setpoints_command']
 ['9279' 'MC3_setpoints_command']
 ['8892' 'MC4_setpoints_command']
 ['3430' 'MCU_status']
 ['6489' 'BMS_onboard_temperatures']
 ['54322' 'BMS_voltages']
 ['26135' 'BMS_detailed_voltages']
 ['12952' 'BMS_temperatures']
 ['8067' 'BMS_detailed_temperatures']
 ['3355' 'BMS_status']
 ['341' 'Dashboard_status']
 ['3397' 'EM_status']
 ['3409' 'EM_measurement']]


In [ ]:
import plotly.graph_objects as go
a = p.parse_BMS_detailed_temperatures(msg_data, msg_id)
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,2], mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,3], mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,4], mode='markers'))
fig.show()

In [42]:
fig = go.Figure()
a = p.parse_MC1_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC2_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC3_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC4_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
fig.show()

In [5]:
for p in Path( '../telemetry_exe/Raw_Data/' ).rglob( '*.csv' ):
    vp.parse_file(p)

In [8]:
data_dict, csv_paths = vp.parse_csv_folder("../telemetry_exe/Raw_Data/")
mat_paths = [os.path.splitext(csv_path)[0] + '.mat' for csv_path in csv_paths]
for i in range(len(mat_paths)):
    scipy.io.savemat(mat_paths[i], {"data": data_dict[csv_paths[i]]})

In [11]:
vp.print_nested_dict(data_dict[csv_paths[0]])

{
    "BMS": {
        "detailed_temps": {
            "ic_00": {
                "temp_1": "[[  0.65   23.73 ]\n [  1.207  23.74 ]\n [  2.057  23.74 ]\n [  3.245  23.74 ]\n [  4.273  23.74 ]\n [  5.11   23.74 ]\n [  6.295  23.74 ]\n [  7.273  23.74 ]\n [ 10.387  23.74 ]\n [ 13.333  23.74 ]\n [ 14.143  23.74 ]\n [ 16.064  23.75 ]\n [ 17.144  23.74 ]\n [ 19.072  23.74 ]\n [ 21.302  23.74 ]\n [ 23.064  23.74 ]\n [ 24.239  23.74 ]\n [ 27.174  23.74 ]\n [ 29.193  23.75 ]\n [ 32.211  23.74 ]\n [ 35.03   23.75 ]\n [ 36.202  23.75 ]\n [ 36.493  23.75 ]\n [ 39.063  23.75 ]\n [ 39.881  23.75 ]\n [ 41.126  23.75 ]\n [ 43.308  23.75 ]\n [ 45.13   23.74 ]\n [ 48.02   23.74 ]\n [ 50.212  23.75 ]\n [ 52.087  23.74 ]\n [ 54.317  23.79 ]\n [ 57.169  23.91 ]\n [ 59.167  23.95 ]\n [ 60.228  23.96 ]\n [ 62.263  23.99 ]\n [ 63.221  24.04 ]\n [ 65.258  24.22 ]\n [ 67.18   24.35 ]\n [ 69.345  24.41 ]\n [ 71.178  24.42 ]\n [ 73.284  24.43 ]\n [ 76.015  24.43 ]\n [ 78.334  24.43 ]\n [ 79.118  24.44 ]\n [ 82.3